In [1]:
import requests
import pandas as pd
import numpy as np

API examination

In [2]:
Source = "https://rickandmortyapi.com/api"
full = requests.get(Source) 
print(full.content) 

b'{"characters":"https://rickandmortyapi.com/api/character","locations":"https://rickandmortyapi.com/api/location","episodes":"https://rickandmortyapi.com/api/episode"}'


## We start with Characters API, as consideration, it will be a total of 3 tables.

In [3]:
Character_url= "https://rickandmortyapi.com/api/character/?page=1"  # Url for first page
Char_response = requests.get(Character_url) # get request for the api

In [4]:
print(Char_response.content[:800]) # We confirm we received the right data

b'{"info":{"count":826,"pages":42,"next":"https://rickandmortyapi.com/api/character/?page=2","prev":null},"results":[{"id":1,"name":"Rick Sanchez","status":"Alive","species":"Human","type":"","gender":"Male","origin":{"name":"Earth (C-137)","url":"https://rickandmortyapi.com/api/location/1"},"location":{"name":"Citadel of Ricks","url":"https://rickandmortyapi.com/api/location/3"},"image":"https://rickandmortyapi.com/api/character/avatar/1.jpeg","episode":["https://rickandmortyapi.com/api/episode/1","https://rickandmortyapi.com/api/episode/2","https://rickandmortyapi.com/api/episode/3","https://rickandmortyapi.com/api/episode/4","https://rickandmortyapi.com/api/episode/5","https://rickandmortyapi.com/api/episode/6","https://rickandmortyapi.com/api/episode/7","https://rickandmortyapi.com/api/e'


In [5]:
res = Char_response.json() # Now we decode using json
df = pd.json_normalize(res, "results") # And turn it to a data frame, since the important information is inside results, we use that to create the df
df.head() # Checking the data

,id,name,status,species,type,gender,image,episode,url,created,origin.name,origin.url,location.name,location.url
0,1,Rick Sanchez,Alive,Human,,Male,https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,Earth (C-137),https://rickandmortyapi.com/api/location/1,Citadel of Ricks,https://rickandmortyapi.com/api/location/3
1,2,Morty Smith,Alive,Human,,Male,https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z,unknown,,Citadel of Ricks,https://rickandmortyapi.com/api/location/3
2,3,Summer Smith,Alive,Human,,Female,https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20
3,4,Beth Smith,Alive,Human,,Female,https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/4,2017-11-04T19:22:43.665Z,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20
4,5,Jerry Smith,Alive,Human,,Male,https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/5,2017-11-04T19:26:56.301Z,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20


## Characters table creation

This API does not contain all the characters at once, there are 42 different pages (url) for all the characters, a total of 826

For that reason, we will iterate in all pages and store each df in a dictionary

In [6]:
Char = {} # Dictionary 
for i in range(1,43):
    url= "https://rickandmortyapi.com/api/character/?page={}".format(i)  # This will iterate in all possible url
    response = requests.get(url) # get request for the API
    res = response.json() # Now we decode using json
    df = pd.json_normalize(res, "results") # An turn it to a data frame
    Char.update({i:df}) # Dictionary is updated
    

In [7]:
new = pd.concat(list(Char.values()), axis=0) # Now is time to concat all the df stored in dictionary
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 826 entries, 0 to 5
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             826 non-null    int64 
 1   name           826 non-null    object
 2   status         826 non-null    object
 3   species        826 non-null    object
 4   type           826 non-null    object
 5   gender         826 non-null    object
 6   image          826 non-null    object
 7   episode        826 non-null    object
 8   url            826 non-null    object
 9   created        826 non-null    object
 10  origin.name    826 non-null    object
 11  origin.url     826 non-null    object
 12  location.name  826 non-null    object
 13  location.url   826 non-null    object
dtypes: int64(1), object(13)
memory usage: 96.8+ KB


In [8]:
new.tail()

,id,name,status,species,type,gender,image,episode,url,created,origin.name,origin.url,location.name,location.url
1,822,Young Jerry,unknown,Human,,Male,https://rickandmortyapi.com/api/character/avat...,[https://rickandmortyapi.com/api/episode/51],https://rickandmortyapi.com/api/character/822,2021-11-02T17:18:31.934Z,Earth (Unknown dimension),https://rickandmortyapi.com/api/location/30,Earth (Unknown dimension),https://rickandmortyapi.com/api/location/30
2,823,Young Beth,unknown,Human,,Female,https://rickandmortyapi.com/api/character/avat...,[https://rickandmortyapi.com/api/episode/51],https://rickandmortyapi.com/api/character/823,2021-11-02T17:19:00.951Z,Earth (Unknown dimension),https://rickandmortyapi.com/api/location/30,Earth (Unknown dimension),https://rickandmortyapi.com/api/location/30
3,824,Young Beth,unknown,Human,,Female,https://rickandmortyapi.com/api/character/avat...,[https://rickandmortyapi.com/api/episode/51],https://rickandmortyapi.com/api/character/824,2021-11-02T17:19:47.957Z,Earth (Unknown dimension),https://rickandmortyapi.com/api/location/30,Earth (Unknown dimension),https://rickandmortyapi.com/api/location/30
4,825,Young Jerry,unknown,Human,,Male,https://rickandmortyapi.com/api/character/avat...,[https://rickandmortyapi.com/api/episode/51],https://rickandmortyapi.com/api/character/825,2021-11-02T17:20:14.305Z,Earth (Unknown dimension),https://rickandmortyapi.com/api/location/30,Earth (Unknown dimension),https://rickandmortyapi.com/api/location/30
5,826,Butter Robot,Alive,Robot,Passing Butter Robot,Genderless,https://rickandmortyapi.com/api/character/avat...,[https://rickandmortyapi.com/api/episode/9],https://rickandmortyapi.com/api/character/826,2021-11-02T17:24:37.458Z,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20


In [9]:
new.to_csv('characters.csv')
cha_csv = pd.read_csv('characters.csv')  
cha_csv = cha_csv.drop(columns = ["Unnamed: 0"])
cha_csv = cha_csv.fillna("No info")

In [10]:
cha_csv.head()

,id,name,status,species,type,gender,image,episode,url,created,origin.name,origin.url,location.name,location.url
0,1,Rick Sanchez,Alive,Human,No info,Male,https://rickandmortyapi.com/api/character/avat...,"['https://rickandmortyapi.com/api/episode/1', ...",https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,Earth (C-137),https://rickandmortyapi.com/api/location/1,Citadel of Ricks,https://rickandmortyapi.com/api/location/3
1,2,Morty Smith,Alive,Human,No info,Male,https://rickandmortyapi.com/api/character/avat...,"['https://rickandmortyapi.com/api/episode/1', ...",https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z,unknown,No info,Citadel of Ricks,https://rickandmortyapi.com/api/location/3
2,3,Summer Smith,Alive,Human,No info,Female,https://rickandmortyapi.com/api/character/avat...,"['https://rickandmortyapi.com/api/episode/6', ...",https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20
3,4,Beth Smith,Alive,Human,No info,Female,https://rickandmortyapi.com/api/character/avat...,"['https://rickandmortyapi.com/api/episode/6', ...",https://rickandmortyapi.com/api/character/4,2017-11-04T19:22:43.665Z,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20
4,5,Jerry Smith,Alive,Human,No info,Male,https://rickandmortyapi.com/api/character/avat...,"['https://rickandmortyapi.com/api/episode/6', ...",https://rickandmortyapi.com/api/character/5,2017-11-04T19:26:56.301Z,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20,Earth (Replacement Dimension),https://rickandmortyapi.com/api/location/20


## Connection to MySql Database

In [11]:
import mysql.connector
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  passwd="*****",
  database = "rick_morty",
  auth_plugin='mysql_native_password'
)
cursor = mydb.cursor()
print(mydb) 

### Table creation in mysql

In [12]:
cursor.execute("CREATE TABLE characters(id int,name varchar(255),status varchar(255),species varchar(255),type varchar(255),gender varchar(255),image varchar(255),episode LONGTEXT,url varchar(255),created varchar(255),`origin.name` varchar(255),`origin.url` varchar(255),`location.name` varchar(255),`location.url` varchar(255))")

In [14]:
  for i,row in cha_csv.iterrows():
            #here %S means string values 
            sql = "INSERT INTO rick_morty.characters VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            # the connection is not auto committed by default, so we must commit to save our changes
            mydb.commit()

## Location table creation

In [15]:
Location_url= "https://rickandmortyapi.com/api/location"  # Url for first page
Loc_response = requests.get(Location_url) # get request for the api
print(Loc_response.content[:800]) # We confirm we received the right data

b'{"info":{"count":126,"pages":7,"next":"https://rickandmortyapi.com/api/location?page=2","prev":null},"results":[{"id":1,"name":"Earth (C-137)","type":"Planet","dimension":"Dimension C-137","residents":["https://rickandmortyapi.com/api/character/38","https://rickandmortyapi.com/api/character/45","https://rickandmortyapi.com/api/character/71","https://rickandmortyapi.com/api/character/82","https://rickandmortyapi.com/api/character/83","https://rickandmortyapi.com/api/character/92","https://rickandmortyapi.com/api/character/112","https://rickandmortyapi.com/api/character/114","https://rickandmortyapi.com/api/character/116","https://rickandmortyapi.com/api/character/117","https://rickandmortyapi.com/api/character/120","https://rickandmortyapi.com/api/character/127","https://rickandmortyapi.com'


Similar situation with characters, more than one page

In [16]:
Loc = {} # Dictionary 
for i in range(1,8):
    url= "https://rickandmortyapi.com/api/location/?page={}".format(i)  # This will iterate in all possible url
    response = requests.get(url) # get request for the API
    res = response.json() # Now we decode using json
    df = pd.json_normalize(res, "results") # An turn it to a data frame
    Loc.update({i:df}) # Dictionary is updated
    

In [17]:
Locations = pd.concat(list(Loc.values()), axis=0) # Now is time to concat all the df stored in dictionary
Locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 0 to 5
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         126 non-null    int64 
 1   name       126 non-null    object
 2   type       126 non-null    object
 3   dimension  126 non-null    object
 4   residents  126 non-null    object
 5   url        126 non-null    object
 6   created    126 non-null    object
dtypes: int64(1), object(6)
memory usage: 7.9+ KB


In [18]:
Locations.head()

,id,name,type,dimension,residents,url,created
0,1,Earth (C-137),Planet,Dimension C-137,"[https://rickandmortyapi.com/api/character/38,...",https://rickandmortyapi.com/api/location/1,2017-11-10T12:42:04.162Z
1,2,Abadango,Cluster,unknown,[https://rickandmortyapi.com/api/character/6],https://rickandmortyapi.com/api/location/2,2017-11-10T13:06:38.182Z
2,3,Citadel of Ricks,Space station,unknown,"[https://rickandmortyapi.com/api/character/8, ...",https://rickandmortyapi.com/api/location/3,2017-11-10T13:08:13.191Z
3,4,Worldender's lair,Planet,unknown,"[https://rickandmortyapi.com/api/character/10,...",https://rickandmortyapi.com/api/location/4,2017-11-10T13:08:20.569Z
4,5,Anatomy Park,Microverse,Dimension C-137,"[https://rickandmortyapi.com/api/character/12,...",https://rickandmortyapi.com/api/location/5,2017-11-10T13:08:46.060Z


In [19]:
Locations.tail()

,id,name,type,dimension,residents,url,created
1,122,Avian Planet,Planet,Replacement Dimension,[https://rickandmortyapi.com/api/character/792...,https://rickandmortyapi.com/api/location/122,2021-10-26T12:19:52.957Z
2,123,Normal Size Bug Dimension,Dimension,,[https://rickandmortyapi.com/api/character/795...,https://rickandmortyapi.com/api/location/123,2021-11-02T13:03:21.307Z
3,124,Slartivart,Planet,Replacement Dimension,[https://rickandmortyapi.com/api/character/797],https://rickandmortyapi.com/api/location/124,2021-11-02T13:07:27.619Z
4,125,Rick and Two Crows Planet,Planet,Replacement Dimension,[https://rickandmortyapi.com/api/character/809...,https://rickandmortyapi.com/api/location/125,2021-11-02T13:50:55.588Z
5,126,Rick's Memories,Memory,,[https://rickandmortyapi.com/api/character/815...,https://rickandmortyapi.com/api/location/126,2021-11-02T15:18:57.987Z


### Loading table to MySql

In [20]:
Locations.to_csv('Locations.csv') # Locations table is stored
loc_csv = pd.read_csv('Locations.csv')  
loc_csv = loc_csv.drop(columns = ["Unnamed: 0"])
loc_csv = loc_csv.fillna("No info")

In [21]:
cursor.execute("CREATE TABLE locations(id int,name varchar(255),type varchar(255),dimension varchar(255), residents LONGTEXT, url varchar(255), created varchar(255))")

In [22]:
for i,row in loc_csv.iterrows():
            #here %S means string values 
            sql = "INSERT INTO rick_morty.locations VALUES (%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            # the connection is not auto committed by default, so we must commit to save our changes
            mydb.commit()

## Episode table creation

In [23]:
Episode_url= "https://rickandmortyapi.com/api/episode"  # Url for first page
Ep_response = requests.get(Episode_url) # get request for the api
print(Ep_response.content[:800]) # We confirm we received the right data

b'{"info":{"count":51,"pages":3,"next":"https://rickandmortyapi.com/api/episode?page=2","prev":null},"results":[{"id":1,"name":"Pilot","air_date":"December 2, 2013","episode":"S01E01","characters":["https://rickandmortyapi.com/api/character/1","https://rickandmortyapi.com/api/character/2","https://rickandmortyapi.com/api/character/35","https://rickandmortyapi.com/api/character/38","https://rickandmortyapi.com/api/character/62","https://rickandmortyapi.com/api/character/92","https://rickandmortyapi.com/api/character/127","https://rickandmortyapi.com/api/character/144","https://rickandmortyapi.com/api/character/158","https://rickandmortyapi.com/api/character/175","https://rickandmortyapi.com/api/character/179","https://rickandmortyapi.com/api/character/181","https://rickandmortyapi.com/api/cha'


In [24]:
Ep = {} # Dictionary 
for i in range(1,4):
    url= "https://rickandmortyapi.com/api/episode/?page={}".format(i)  # This will iterate in all possible url
    response = requests.get(url) # get request for the API
    res = response.json() # Now we decode using json
    df = pd.json_normalize(res, "results") # An turn it to a data frame
    Ep.update({i:df}) # Dictionary is updated
    

In [25]:
Episodes = pd.concat(list(Ep.values()), axis=0) # Now is time to concat all the df stored in dictionary
Episodes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 10
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          51 non-null     int64 
 1   name        51 non-null     object
 2   air_date    51 non-null     object
 3   episode     51 non-null     object
 4   characters  51 non-null     object
 5   url         51 non-null     object
 6   created     51 non-null     object
dtypes: int64(1), object(6)
memory usage: 3.2+ KB


In [26]:
Episodes.head()

,id,name,air_date,episode,characters,url,created
0,1,Pilot,"December 2, 2013",S01E01,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/1,2017-11-10T12:56:33.798Z
1,2,Lawnmower Dog,"December 9, 2013",S01E02,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/2,2017-11-10T12:56:33.916Z
2,3,Anatomy Park,"December 16, 2013",S01E03,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/3,2017-11-10T12:56:34.022Z
3,4,M. Night Shaym-Aliens!,"January 13, 2014",S01E04,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/4,2017-11-10T12:56:34.129Z
4,5,Meeseeks and Destroy,"January 20, 2014",S01E05,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/5,2017-11-10T12:56:34.236Z


In [27]:
Episodes.tail()

,id,name,air_date,episode,characters,url,created
6,47,Rick & Morty's Thanksploitation Spectacular,"July 25, 2021",S05E06,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/47,2021-10-15T17:00:24.104Z
7,48,Gotron Jerrysis Rickvangelion,"August 1, 2021",S05E07,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/48,2021-10-15T17:00:24.104Z
8,49,Rickternal Friendshine of the Spotless Mort,"August 8, 2021",S05E08,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/49,2021-10-15T17:00:24.104Z
9,50,Forgetting Sarick Mortshall,"September 5, 2021",S05E09,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/50,2021-10-15T17:00:24.105Z
10,51,Rickmurai Jack,"September 5, 2021",S05E10,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/51,2021-10-15T17:00:24.105Z


### Loading table to MySql

In [30]:
Episodes.to_csv('Episodes.csv') # Locations table is stored
ep_csv = pd.read_csv('Episodes.csv')  
ep_csv = ep_csv.drop(columns = ["Unnamed: 0"])
ep_csv = ep_csv.fillna("No info")

In [31]:
cursor.execute("CREATE TABLE episodes(id int,name varchar(255),air_date varchar(255),episode varchar(255), characters LONGTEXT, url varchar(255), created varchar(255))")

In [32]:
for i,row in ep_csv.iterrows():
            #here %S means string values 
            sql = "INSERT INTO rick_morty.episodes VALUES (%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            # the connection is not auto committed by default, so we must commit to save our changes
            mydb.commit()